<a href="https://colab.research.google.com/github/MehrdadJalali-AI/LotusEffectAlgorithm/blob/main/MLEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import gamma

# Define the 14 benchmark functions
def ackley(x):
    a, b, c = 20, 0.2, 2 * np.pi
    d = len(x)
    return -a * np.exp(-b * np.sqrt(np.sum(x**2) / d)) - np.exp(np.sum(np.cos(c * x)) / d) + a + np.exp(1)

def six_hump(x):
    x1, x2 = x[0], x[1]
    return (4 - 2.1 * x1**2 + (x1**4)/3) * x1**2 + x1*x2 + (-4 + 4*x2**2) * x2**2

def decreasing_maxima(x):
    x = np.maximum(x, 1e-10)  # Avoid invalid power operations
    return np.exp(-2 * np.log(2) * ((x - 0.08)/0.854)**2) * np.sin(5 * np.pi * ((x**(3/4)) - 0.05))**6

def equal_maxima(x):
    return np.sin(5 * np.pi * x)**6

def himmelblau(x):
    x1, x2 = x[0], x[1]
    return (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2

def modified_rastrigin(x):
    return np.sum(10 + 9 * np.cos(2 * np.pi * (x - 0.5)))

def vincent(x):
    x = np.maximum(x, 1e-10)  # Avoid log(0) errors
    return (1 / len(x)) * np.sum(np.sin(10 * np.log(x)))

def shubert(x):
    x1, x2 = x[0], x[1]
    return np.prod([np.sum(i * np.cos((i + 1) * xj + i)) for i in range(1, 6) for xj in [x1, x2]])

def composition_function_1(x):
    return np.sum(x**2) + np.sum(np.cos(2 * np.pi * x))

def composition_function_2(x):
    return np.sum(x**2) + np.sum(np.sin(2 * np.pi * x))

def five_uneven_peak_trap(x):
    return np.sin(5 * np.pi * x)**6

def expanded_equal_maxima(x):
    return np.sum(np.sin(5 * np.pi * x)**6)

def expanded_uneven_maxima(x):
    return np.sum(np.sin(5 * np.pi * (x**(3/4) - 0.05))**6)

def modified_vincent(x):
    return np.sum(np.sin(10 * np.log(x)) + 1)

# Store all functions in a dictionary
benchmark_functions = {
    "Ackley": ackley,
    "Six Hump": six_hump,
    "Decreasing Maxima": decreasing_maxima,
    "Equal Maxima": equal_maxima,
    "Himmelblau": himmelblau,
    "Modified Rastrigin": modified_rastrigin,
    "Vincent": vincent,
    "Shubert": shubert,
    "Composition Function 1": composition_function_1,
    "Composition Function 2": composition_function_2,
    "Five Uneven Peak Trap": five_uneven_peak_trap,
    "Expanded Equal Maxima": expanded_equal_maxima,
    "Expanded Uneven Maxima": expanded_uneven_maxima,
    "Modified Vincent": modified_vincent
}

# Define the Lotus Effect Algorithm (LEA) with MBRO-inspired enhancements
class LotusEffectAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_function_evaluations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_function_evaluations = max_function_evaluations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))
        self.best_solution = self.population[0].copy()
        self.best_fitness = float('inf')
        self.function_evaluations = 0
        self.alpha = 0.5
        self.beta = 1.5
        self.memory = []

    def levy_flight(self):
        sigma = (gamma(1 + self.beta) * np.sin(np.pi * self.beta / 2) /
                 (gamma((1 + self.beta) / 2) * self.beta * 2 ** ((self.beta - 1) / 2))) ** (1 / self.beta)
        u = np.random.normal(0, sigma, size=self.dimensions)
        v = np.random.normal(0, 1, size=self.dimensions)
        step = u / np.abs(v) ** (1 / self.beta)
        return 0.01 * step

    def update_positions(self):
        for i in range(self.population_size):
            if self.function_evaluations >= self.max_function_evaluations:
                return
            step = self.alpha * (self.best_solution - self.population[i]) + self.levy_flight()
            self.population[i] += step
            self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
            self.function_evaluations += 1

    def evaluate_fitness(self):
        for i in range(self.population_size):
            if self.function_evaluations >= self.max_function_evaluations:
                return
            fitness = self.fitness_function(self.population[i])
            self.function_evaluations += 1
            if np.isscalar(fitness):
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()
                    self.memory.append(self.best_solution)
            else:
                if np.all(fitness < self.best_fitness):
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()
                    self.memory.append(self.best_solution)

    def optimize(self):
        while self.function_evaluations < self.max_function_evaluations:
            self.update_positions()
            self.evaluate_fitness()
        return self.best_solution, self.best_fitness, self.function_evaluations

# Running evaluations for all 14 functions
num_runs = 10
max_function_evaluations = 100000

evaluation_counts = []
optimal_values = []
success_rates = []
success_performance = []
maximum_peak_ratios = []

for name, func in benchmark_functions.items():
    best_fitnesses = []
    function_evals = []
    for _ in range(num_runs):
        lea = LotusEffectAlgorithm(
            population_size=30, dimensions=2, lower_bound=-5, upper_bound=5,
            max_function_evaluations=max_function_evaluations, fitness_function=func
        )
        _, best_fitness, function_evaluations = lea.optimize()
        best_fitnesses.append(best_fitness)
        function_evals.append(function_evaluations)

    evaluation_counts.append(np.mean(function_evals))
    optimal_values.append(np.mean(best_fitnesses))
    success_rate = np.mean(np.array(best_fitnesses) <= 0.01)
    success_rates.append(success_rate)
    success_performance.append(np.sum(function_evals) / (num_runs * success_rates[-1]) if success_rates[-1] > 0 else np.inf)
    maximum_peak_ratios.append(np.mean(best_fitnesses))

# Create results table
df_results = pd.DataFrame({
    "Function": list(benchmark_functions.keys()),
    "Average Function Evaluations": evaluation_counts,
    "LEA Average Best Fitness": optimal_values,
    "Success Rate (SR)": success_rates,
    "Success Performance (SP)": success_performance,
    "Maximum Peak Ratio (MPR)": maximum_peak_ratios
})

# Save results to CSV
df_results.to_csv("LEA_Results.csv", index=False)

# Display the results table
print(df_results)
